In [1]:
from predictor import predict_word
import pandas as pd

## PIPELINE
from transformers import pipeline, AutoTokenizer
import torch
import torch.nn.functional as F

/Users/samet/opt/anaconda3/envs/hf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('masked_df.csv', index_col=False).drop('id', axis=1)

In [3]:
df

,word,snippet
0,Info,"Path:filepath.Join(dir,f.Name()),<mask>:f,})}"
1,From,"ifmock==nil{panic(""mockmustnotbenil.UsemyMock:..."
2,inputs,"switchlen(f.inputs){case1:source[""input""]=f.in..."
3,[],returnasRuntimeObjects(parseRules(client.Clust...
4,for,"<mask>_,o:=rangeoo{varpodv1.Poderr=runtime.Def..."
...,...,...
24584,hydrate,func(s*Sanitizer)hydrate(parent*tview.TreeNode...
24585,FullScreen,returnnil}func(d*Details)toggle<mask>Cmd(evt*t...
24586,TableModel,"func(t*mock<mask>)Delete(context.Context,strin..."
24587,InputHandler,"returnc.Wrap<mask>(func(event*tcell.EventKey,s..."


In [4]:
sample = df['snippet'].iloc[:10]

In [5]:

def predict_word(sequence:str) -> "list[str]" :
    '''Predict the masked words. Mask token:<mask>

    Args:
        sequence (list[str]): Sequence of code snippets include masked tokens
        return_sequence (bool): True if you want whole sequence, false if you only want predicted word
    Returns:
        Returns predicted words 
    '''
    prediction = []
    fill_mask = pipeline(
    "fill-mask",
    model="huggingface/CodeBERTa-small-v1",
    # tokenizer="huggingface/CodeBERTa-small-v1",
    tokenizer = AutoTokenizer.from_pretrained("huggingface/CodeBERTa-small-v1", model_max_len=512, padding=True, truncation='drop_rows_to_fit')

)
    outputs = fill_mask(sequence)
    for output in outputs:
        prediction.append(output)

    return prediction

In [20]:
labels = df['word'].iloc[:10]
labels

0      Info
1      From
2    inputs
3        []
4       for
5    status
6    return
7     final
8       nil
9       err
Name: word, dtype: object

In [21]:
result = []
for text in sample:
    result.append(predict_word(text))

In [22]:

limit = 5
sum_pred = []
for label, pred_list in zip(labels,result):
    preds = []
    for idx, pred in enumerate(pred_list):
        if idx > limit-1:
            break
        preds.append(pred['token_str'])
    if label in preds:
        sum_pred.append(1)
    else:
        sum_pred.append(0)

In [23]:
len(sum_pred)

10

In [24]:
import numpy as np

In [25]:
pred_np =  np.array(sum_pred)

In [26]:
pred_np.sum()

3

In [27]:
preds

['err', 'conn', 'log', 'error', 'e']